In [1]:
#!pip install pyspark
#!pip install findspark

In [1]:
import findspark
findspark.init()
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

In [93]:
%run "./Setup"

+---------+--------------------+-------------------+---------+-------------------+
|course_id|        course_title|course_published_dt|is_active|    last_updated_ts|
+---------+--------------------+-------------------+---------+-------------------+
|        1|    Mastering Python|         2021-01-14|     true|2021-02-18 16:57:25|
|        2|Data Engineering ...|         2021-02-10|     true|2021-03-05 12:07:33|
|        3|   Mastering Pyspark|         2021-01-07|     true|2021-04-06 10:05:42|
|        4|      AWS Essentials|         2021-03-19|    false|2021-04-10 02:25:36|
|        5|          Docker 101|         2021-02-28|     true|2021-03-21 07:18:52|
+---------+--------------------+-------------------+---------+-------------------+

+-------+---------------+--------------+--------------------+
|user_id|user_first_name|user_last_name|          user_email|
+-------+---------------+--------------+--------------------+
|      1|         Sandra|        Karpov|    skarpov0@ovh.net|
|   

* Get the user details who have enrolled for the courses.
  * Need to join **users_df** and **course_enrolments_df**.
  * Here are the fields that needs to be displayed.
    * All fields from `users_df`
    * `course_id` and `course_enrolment_id` from `course_enrolments`

In [30]:
users_df.show()

+-------+---------------+--------------+--------------------+
|user_id|user_first_name|user_last_name|          user_email|
+-------+---------------+--------------+--------------------+
|      1|         Sandra|        Karpov|    skarpov0@ovh.net|
|      2|           Kari|        Dearth|kdearth1@so-net.n...|
|      3|         Joanna|      Spennock|jspennock2@redcro...|
|      4|         Hirsch|       Conaboy|hconaboy3@barnesa...|
|      5|         Loreen|         Malin|lmalin4@independe...|
|      6|           Augy|      Christon|  achriston5@mlb.com|
|      7|         Trudey|       Choupin|     tchoupin6@de.vu|
|      8|         Nadine|     Grimsdell|ngrimsdell7@sohu.com|
|      9|        Vassily|         Tamas|vtamas8@businessw...|
|     10|          Wells|      Simpkins|wsimpkins9@amazon...|
+-------+---------------+--------------+--------------------+



In [65]:
users_df.rdd.getNumPartitions()

4

In [33]:
users_df = users_df.repartition(6)

In [34]:
users_df.show()

+-------+---------------+--------------+--------------------+
|user_id|user_first_name|user_last_name|          user_email|
+-------+---------------+--------------+--------------------+
|      6|           Augy|      Christon|  achriston5@mlb.com|
|      8|         Nadine|     Grimsdell|ngrimsdell7@sohu.com|
|      2|           Kari|        Dearth|kdearth1@so-net.n...|
|      1|         Sandra|        Karpov|    skarpov0@ovh.net|
|      9|        Vassily|         Tamas|vtamas8@businessw...|
|      3|         Joanna|      Spennock|jspennock2@redcro...|
|      7|         Trudey|       Choupin|     tchoupin6@de.vu|
|      4|         Hirsch|       Conaboy|hconaboy3@barnesa...|
|      5|         Loreen|         Malin|lmalin4@independe...|
|     10|          Wells|      Simpkins|wsimpkins9@amazon...|
+-------+---------------+--------------+--------------------+



In [41]:
users_df.coalesce(4).show()

+-------+---------------+--------------+--------------------+
|user_id|user_first_name|user_last_name|          user_email|
+-------+---------------+--------------+--------------------+
|      6|           Augy|      Christon|  achriston5@mlb.com|
|      8|         Nadine|     Grimsdell|ngrimsdell7@sohu.com|
|      4|         Hirsch|       Conaboy|hconaboy3@barnesa...|
|      5|         Loreen|         Malin|lmalin4@independe...|
|     10|          Wells|      Simpkins|wsimpkins9@amazon...|
|      9|        Vassily|         Tamas|vtamas8@businessw...|
|      1|         Sandra|        Karpov|    skarpov0@ovh.net|
|      2|           Kari|        Dearth|kdearth1@so-net.n...|
|      3|         Joanna|      Spennock|jspennock2@redcro...|
|      7|         Trudey|       Choupin|     tchoupin6@de.vu|
+-------+---------------+--------------+--------------------+



In [69]:
users_df.coalesce(1).sort("user_id").rdd.getNumPartitions()

1

In [97]:
users_df = users_df.repartition(1)
users_df.rdd.getNumPartitions()

1

In [98]:
users_df.sort("user_email").explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- Sort [user_email#906 ASC NULLS FIRST], true, 0
   +- Exchange SinglePartition, REPARTITION_BY_NUM, [id=#1808]
      +- Scan ExistingRDD[user_id#903L,user_first_name#904,user_last_name#905,user_email#906]




In [49]:
users_df.orderBy("user_email").rdd.getNumPartitions()

1

In [94]:
users_df_sort = users_df.orderBy("user_email")

In [95]:
users_df_sort.explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- Sort [user_email#906 ASC NULLS FIRST], true, 0
   +- Exchange rangepartitioning(user_email#906 ASC NULLS FIRST, 200), ENSURE_REQUIREMENTS, [id=#1773]
      +- Scan ExistingRDD[user_id#903L,user_first_name#904,user_last_name#905,user_email#906]




In [96]:
users_df_sort.show()

+-------+---------------+--------------+--------------------+
|user_id|user_first_name|user_last_name|          user_email|
+-------+---------------+--------------+--------------------+
|      6|           Augy|      Christon|  achriston5@mlb.com|
|      4|         Hirsch|       Conaboy|hconaboy3@barnesa...|
|      3|         Joanna|      Spennock|jspennock2@redcro...|
|      2|           Kari|        Dearth|kdearth1@so-net.n...|
|      5|         Loreen|         Malin|lmalin4@independe...|
|      8|         Nadine|     Grimsdell|ngrimsdell7@sohu.com|
|      1|         Sandra|        Karpov|    skarpov0@ovh.net|
|      7|         Trudey|       Choupin|     tchoupin6@de.vu|
|      9|        Vassily|         Tamas|vtamas8@businessw...|
|     10|          Wells|      Simpkins|wsimpkins9@amazon...|
+-------+---------------+--------------+--------------------+



In [53]:
users_df_sort.show()

+-------+---------------+--------------+--------------------+
|user_id|user_first_name|user_last_name|          user_email|
+-------+---------------+--------------+--------------------+
|      6|           Augy|      Christon|  achriston5@mlb.com|
|      4|         Hirsch|       Conaboy|hconaboy3@barnesa...|
|      3|         Joanna|      Spennock|jspennock2@redcro...|
|      2|           Kari|        Dearth|kdearth1@so-net.n...|
|      5|         Loreen|         Malin|lmalin4@independe...|
|      8|         Nadine|     Grimsdell|ngrimsdell7@sohu.com|
|      1|         Sandra|        Karpov|    skarpov0@ovh.net|
|      7|         Trudey|       Choupin|     tchoupin6@de.vu|
|      9|        Vassily|         Tamas|vtamas8@businessw...|
|     10|          Wells|      Simpkins|wsimpkins9@amazon...|
+-------+---------------+--------------+--------------------+



In [5]:
users_df. \
    join(course_enrolments_df, users_df.user_id == course_enrolments_df.user_id). \
    show()

+-------+---------------+--------------+--------------------+-------------------+-------+---------+----------+
|user_id|user_first_name|user_last_name|          user_email|course_enrolment_id|user_id|course_id|price_paid|
+-------+---------------+--------------+--------------------+-------------------+-------+---------+----------+
|      3|         Joanna|      Spennock|jspennock2@redcro...|                 10|      3|        3|     10.99|
|      3|         Joanna|      Spennock|jspennock2@redcro...|                 12|      3|        2|      9.99|
|      4|         Hirsch|       Conaboy|hconaboy3@barnesa...|                  7|      4|        5|     10.99|
|      4|         Hirsch|       Conaboy|hconaboy3@barnesa...|                 14|      4|        3|     10.99|
|      5|         Loreen|         Malin|lmalin4@independe...|                  2|      5|        2|      9.99|
|      5|         Loreen|         Malin|lmalin4@independe...|                  6|      5|        5|     10.99|
|

In [4]:
users_df.rdd.getNumPartitions()

4

In [6]:
# as both data frames have user id using same name, we can pass column name as string as well
users_df. \
    join(course_enrolments_df, 'user_id'). \
    show()

+-------+---------------+--------------+--------------------+-------------------+---------+----------+
|user_id|user_first_name|user_last_name|          user_email|course_enrolment_id|course_id|price_paid|
+-------+---------------+--------------+--------------------+-------------------+---------+----------+
|      3|         Joanna|      Spennock|jspennock2@redcro...|                 10|        3|     10.99|
|      3|         Joanna|      Spennock|jspennock2@redcro...|                 12|        2|      9.99|
|      4|         Hirsch|       Conaboy|hconaboy3@barnesa...|                  7|        5|     10.99|
|      4|         Hirsch|       Conaboy|hconaboy3@barnesa...|                 14|        3|     10.99|
|      5|         Loreen|         Malin|lmalin4@independe...|                  2|        2|      9.99|
|      5|         Loreen|         Malin|lmalin4@independe...|                  6|        5|     10.99|
|      5|         Loreen|         Malin|lmalin4@independe...|            

In [7]:
# Get all columns from users_df and course_id as well as course_enrolment_id from course_enrolments
users_df. \
    join(course_enrolments_df, users_df.user_id == course_enrolments_df.user_id). \
    select(users_df['*'], course_enrolments_df['course_id'], course_enrolments_df['course_enrolment_id']). \
    show()

+-------+---------------+--------------+--------------------+---------+-------------------+
|user_id|user_first_name|user_last_name|          user_email|course_id|course_enrolment_id|
+-------+---------------+--------------+--------------------+---------+-------------------+
|      3|         Joanna|      Spennock|jspennock2@redcro...|        3|                 10|
|      3|         Joanna|      Spennock|jspennock2@redcro...|        2|                 12|
|      4|         Hirsch|       Conaboy|hconaboy3@barnesa...|        5|                  7|
|      4|         Hirsch|       Conaboy|hconaboy3@barnesa...|        3|                 14|
|      5|         Loreen|         Malin|lmalin4@independe...|        2|                  2|
|      5|         Loreen|         Malin|lmalin4@independe...|        5|                  6|
|      5|         Loreen|         Malin|lmalin4@independe...|        2|                 13|
|      7|         Trudey|       Choupin|     tchoupin6@de.vu|        5|         

In [8]:
# using alias
users_df.alias('u'). \
    join(course_enrolments_df.alias('ce'), users_df.user_id == course_enrolments_df.user_id). \
    select('u.*', 'course_id', 'course_enrolment_id'). \
    show()

+-------+---------------+--------------+--------------------+---------+-------------------+
|user_id|user_first_name|user_last_name|          user_email|course_id|course_enrolment_id|
+-------+---------------+--------------+--------------------+---------+-------------------+
|      3|         Joanna|      Spennock|jspennock2@redcro...|        3|                 10|
|      3|         Joanna|      Spennock|jspennock2@redcro...|        2|                 12|
|      4|         Hirsch|       Conaboy|hconaboy3@barnesa...|        5|                  7|
|      4|         Hirsch|       Conaboy|hconaboy3@barnesa...|        3|                 14|
|      5|         Loreen|         Malin|lmalin4@independe...|        2|                  2|
|      5|         Loreen|         Malin|lmalin4@independe...|        5|                  6|
|      5|         Loreen|         Malin|lmalin4@independe...|        2|                 13|
|      7|         Trudey|       Choupin|     tchoupin6@de.vu|        5|         

## Left Join

In [9]:
users_df. \
    join(course_enrolments_df, users_df.user_id == course_enrolments_df.user_id, 'left'). \
    show()

+-------+---------------+--------------+--------------------+-------------------+-------+---------+----------+
|user_id|user_first_name|user_last_name|          user_email|course_enrolment_id|user_id|course_id|price_paid|
+-------+---------------+--------------+--------------------+-------------------+-------+---------+----------+
|      1|         Sandra|        Karpov|    skarpov0@ovh.net|               null|   null|     null|      null|
|      2|           Kari|        Dearth|kdearth1@so-net.n...|               null|   null|     null|      null|
|      3|         Joanna|      Spennock|jspennock2@redcro...|                 10|      3|        3|     10.99|
|      3|         Joanna|      Spennock|jspennock2@redcro...|                 12|      3|        2|      9.99|
|      4|         Hirsch|       Conaboy|hconaboy3@barnesa...|                  7|      4|        5|     10.99|
|      4|         Hirsch|       Conaboy|hconaboy3@barnesa...|                 14|      4|        3|     10.99|
|

In [10]:
# left or left_outer or leftouter are same.

users_df. \
    join(course_enrolments_df, users_df.user_id == course_enrolments_df.user_id, 'left_outer'). \
    show()

+-------+---------------+--------------+--------------------+-------------------+-------+---------+----------+
|user_id|user_first_name|user_last_name|          user_email|course_enrolment_id|user_id|course_id|price_paid|
+-------+---------------+--------------+--------------------+-------------------+-------+---------+----------+
|      1|         Sandra|        Karpov|    skarpov0@ovh.net|               null|   null|     null|      null|
|      2|           Kari|        Dearth|kdearth1@so-net.n...|               null|   null|     null|      null|
|      3|         Joanna|      Spennock|jspennock2@redcro...|                 10|      3|        3|     10.99|
|      3|         Joanna|      Spennock|jspennock2@redcro...|                 12|      3|        2|      9.99|
|      4|         Hirsch|       Conaboy|hconaboy3@barnesa...|                  7|      4|        5|     10.99|
|      4|         Hirsch|       Conaboy|hconaboy3@barnesa...|                 14|      4|        3|     10.99|
|

In [11]:
users_df. \
    join(course_enrolments_df, users_df.user_id == course_enrolments_df.user_id, 'left'). \
    select(users_df['*'], course_enrolments_df['course_id'], course_enrolments_df['course_enrolment_id']). \
    show()

+-------+---------------+--------------+--------------------+---------+-------------------+
|user_id|user_first_name|user_last_name|          user_email|course_id|course_enrolment_id|
+-------+---------------+--------------+--------------------+---------+-------------------+
|      1|         Sandra|        Karpov|    skarpov0@ovh.net|     null|               null|
|      2|           Kari|        Dearth|kdearth1@so-net.n...|     null|               null|
|      3|         Joanna|      Spennock|jspennock2@redcro...|        3|                 10|
|      3|         Joanna|      Spennock|jspennock2@redcro...|        2|                 12|
|      4|         Hirsch|       Conaboy|hconaboy3@barnesa...|        5|                  7|
|      4|         Hirsch|       Conaboy|hconaboy3@barnesa...|        3|                 14|
|      5|         Loreen|         Malin|lmalin4@independe...|        2|                  2|
|      5|         Loreen|         Malin|lmalin4@independe...|        5|         

In [12]:
# using alias
users_df.alias('u'). \
    join(course_enrolments_df.alias('ce'), users_df.user_id == course_enrolments_df.user_id, 'left'). \
    select('u.*', 'course_id', 'course_enrolment_id'). \
    show()

+-------+---------------+--------------+--------------------+---------+-------------------+
|user_id|user_first_name|user_last_name|          user_email|course_id|course_enrolment_id|
+-------+---------------+--------------+--------------------+---------+-------------------+
|      1|         Sandra|        Karpov|    skarpov0@ovh.net|     null|               null|
|      2|           Kari|        Dearth|kdearth1@so-net.n...|     null|               null|
|      3|         Joanna|      Spennock|jspennock2@redcro...|        3|                 10|
|      3|         Joanna|      Spennock|jspennock2@redcro...|        2|                 12|
|      4|         Hirsch|       Conaboy|hconaboy3@barnesa...|        5|                  7|
|      4|         Hirsch|       Conaboy|hconaboy3@barnesa...|        3|                 14|
|      5|         Loreen|         Malin|lmalin4@independe...|        2|                  2|
|      5|         Loreen|         Malin|lmalin4@independe...|        5|         

## Right Join

In [13]:
course_enrolments_df. \
    join(users_df, users_df.user_id == course_enrolments_df.user_id, 'right'). \
    show()

+-------------------+-------+---------+----------+-------+---------------+--------------+--------------------+
|course_enrolment_id|user_id|course_id|price_paid|user_id|user_first_name|user_last_name|          user_email|
+-------------------+-------+---------+----------+-------+---------------+--------------+--------------------+
|               null|   null|     null|      null|      1|         Sandra|        Karpov|    skarpov0@ovh.net|
|               null|   null|     null|      null|      2|           Kari|        Dearth|kdearth1@so-net.n...|
|                 10|      3|        3|     10.99|      3|         Joanna|      Spennock|jspennock2@redcro...|
|                 12|      3|        2|      9.99|      3|         Joanna|      Spennock|jspennock2@redcro...|
|                  7|      4|        5|     10.99|      4|         Hirsch|       Conaboy|hconaboy3@barnesa...|
|                 14|      4|        3|     10.99|      4|         Hirsch|       Conaboy|hconaboy3@barnesa...|
|

In [14]:
# right or right_outer or rightouter are same.

course_enrolments_df. \
    join(users_df, users_df.user_id == course_enrolments_df.user_id, 'right_outer'). \
    show()

+-------------------+-------+---------+----------+-------+---------------+--------------+--------------------+
|course_enrolment_id|user_id|course_id|price_paid|user_id|user_first_name|user_last_name|          user_email|
+-------------------+-------+---------+----------+-------+---------------+--------------+--------------------+
|               null|   null|     null|      null|      1|         Sandra|        Karpov|    skarpov0@ovh.net|
|               null|   null|     null|      null|      2|           Kari|        Dearth|kdearth1@so-net.n...|
|                 10|      3|        3|     10.99|      3|         Joanna|      Spennock|jspennock2@redcro...|
|                 12|      3|        2|      9.99|      3|         Joanna|      Spennock|jspennock2@redcro...|
|                  7|      4|        5|     10.99|      4|         Hirsch|       Conaboy|hconaboy3@barnesa...|
|                 14|      4|        3|     10.99|      4|         Hirsch|       Conaboy|hconaboy3@barnesa...|
|

## Full Join

In [15]:
users1_df.count()

15

In [16]:
users2_df.count()

13

In [17]:
# outer or full or fullouter or full_outer are same

users1_df. \
    join(users2_df, users1_df.email == users2_df.email, 'full'). \
    show()

+--------------------+----------+----------+-----------+---------------+--------------------+----------+------------+-----------+---------------+
|               email|first_name| last_name|     gender|     ip_address|               email|first_name|   last_name|     gender|     ip_address|
+--------------------+----------+----------+-----------+---------------+--------------------+----------+------------+-----------+---------------+
|afaulconerb@barne...|    Andris| Faulconer|     Female| 109.40.175.103|afaulconerb@barne...|    Andris|   Faulconer|     Female| 109.40.175.103|
| alovett0@nsw.gov.au|   Aundrea|    Lovett|       Male|    62.72.1.143|                null|      null|        null|       null|           null|
|                null|      null|      null|       null|           null|   amillinsg@com.com|  Adelaide|     Millins|     Female| 75.160.220.182|
|bjowling1@spiegel.de|   Bettine|   Jowling|     Female|  26.250.197.47|                null|      null|        null|       

In [18]:
users1_df. \
    join(users2_df, users1_df.email == users2_df.email, 'fullouter'). \
    show()

+--------------------+----------+----------+-----------+---------------+--------------------+----------+------------+-----------+---------------+
|               email|first_name| last_name|     gender|     ip_address|               email|first_name|   last_name|     gender|     ip_address|
+--------------------+----------+----------+-----------+---------------+--------------------+----------+------------+-----------+---------------+
|afaulconerb@barne...|    Andris| Faulconer|     Female| 109.40.175.103|afaulconerb@barne...|    Andris|   Faulconer|     Female| 109.40.175.103|
| alovett0@nsw.gov.au|   Aundrea|    Lovett|       Male|    62.72.1.143|                null|      null|        null|       null|           null|
|                null|      null|      null|       null|           null|   amillinsg@com.com|  Adelaide|     Millins|     Female| 75.160.220.182|
|bjowling1@spiegel.de|   Bettine|   Jowling|     Female|  26.250.197.47|                null|      null|        null|       

In [19]:
users1_df. \
    join(users2_df, users1_df.email == users2_df.email, 'full_outer'). \
    show()

+--------------------+----------+----------+-----------+---------------+--------------------+----------+------------+-----------+---------------+
|               email|first_name| last_name|     gender|     ip_address|               email|first_name|   last_name|     gender|     ip_address|
+--------------------+----------+----------+-----------+---------------+--------------------+----------+------------+-----------+---------------+
|afaulconerb@barne...|    Andris| Faulconer|     Female| 109.40.175.103|afaulconerb@barne...|    Andris|   Faulconer|     Female| 109.40.175.103|
| alovett0@nsw.gov.au|   Aundrea|    Lovett|       Male|    62.72.1.143|                null|      null|        null|       null|           null|
|                null|      null|      null|       null|           null|   amillinsg@com.com|  Adelaide|     Millins|     Female| 75.160.220.182|
|bjowling1@spiegel.de|   Bettine|   Jowling|     Female|  26.250.197.47|                null|      null|        null|       